In [1]:
#该cell运行一次安装成功后无需再运行，不用在意ERROR，重启内核直接运行下一个cell
!pip install transformers -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install --user datasets -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 10.4 MB 353 kB/s eta 0:00:01�██████████          | 7.1 MB 1.5 MB/s eta 0:00:03
     |████████████████████████████████| 471 kB 180 kB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 252 kB/s eta 0:00:01
     |████████████████████████████████| 481 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 194 kB 994 kB/s eta 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.6.0
    Uninstalling fsspec-2021.6.0:
      Successfully uninstalled fsspec-2021.6.0
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 491 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 193 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 9.9 MB/s s eta 0:00:01
     |████████████████████████████████| 133 kB 7.9 MB/s eta 0:00:011
     |████████████████████████████████| 1.6 MB 2.5 M

     |████████████████████████████████| 209 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 216 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 889 kB/s eta 0:00:011
     |████████████████████████████████| 116 kB 140.4 MB/s eta 0:00:01
     |████████████████████████████████| 146 kB 1.6 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.13.2 which is incompatible.


In [1]:
import json
from Model import BertModel
from transformers import AutoTokenizer
import torch
from torch import nn
import re
from datasets import load_dataset
import os

In [2]:
class MyConfig(object):
    """Configuration class to store the configuration of a 'Model'
    """
    def __init__(self, vocab_size_or_config_json_file):
        with open(vocab_size_or_config_json_file, "r") as reader:
            json_config = json.loads(reader.read())
        for key, value in json_config.items():
            self.__dict__[key] = value

In [3]:
def load_file(file_path):
    with open(file_path) as f:
        test_set = f.readlines()

    test_set = [line.strip() for line in test_set]
    return test_set

In [4]:
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    my_config = MyConfig('./bert_config.json')
    my_model = BertModel(my_config, device=device)
    model_dic = torch.load('./pre-trained.bin', map_location=device)
    my_model.load_state_dict(model_dic, strict=False)
    my_model.eval()

    tokenizer = AutoTokenizer.from_pretrained("./tokenizer")
    X = load_file('./input/X_test.txt') 
    inputs = tokenizer(X, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        logits = my_model(inputs['input_ids'])

    predicted_class_id = logits.argmax(-1).tolist()
    pred_txt = ['0' if x == 0 else '1' for x in predicted_class_id]

    with open('output/predict.txt', 'w') as g:
        g.write('\n'.join(pred_txt))

In [5]:
def zip_fun():
    path=os.getcwd()
    newpath=path+"/output/"
    os.chdir(newpath)
    os.system('zip prediction.zip predict.txt')
    os.chdir(path)

In [6]:
if __name__ == '__main__':
    main()
    zip_fun()